## Setup


In [2]:
%%capture
!pip install wandb --upgrade
!pip install tensorflow-addons

In [51]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.layers import Embedding, SimpleRNNCell, GRUCell, Dense, LSTMCell
from tensorflow.keras import Input
import pandas as pd
from numpy import argmax
from math import log
import pprint
import math
import wandb
import os
import io
from wandb.keras import WandbCallback
import time
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# wandb_api = user_secrets.get_secret("wandb_api")

# #wandb.login(key=wandb_api)
# ! wandb login $wandb_api

# os.environ["WANDB_SILENT"] = "true"
wandb.login()

wandb: Currently logged in as: aslan. Use `wandb login --relogin` to force relogin


True

## Download the data


## Configuration


In [4]:
!wget  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xf 'dakshina_dataset_v1.0.tar'
train_file_path = "/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
val_file_path= "/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv"
test_file_path  = "/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv"

--2022-06-20 17:43:06--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.211.128, 173.194.212.128, 173.194.214.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.211.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G  89.6MB/s    in 17s     

2022-06-20 17:43:23 (111 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [5]:
batch_size = 256  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 100000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = train_file_path


## Prepare the data


In [6]:
def processData(filename,input_chars=set(),target_chars=set()):
  input=[]
  target=[]
  with open(filename, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
  for line in lines[:len(lines) -1]:
      t_text,i_text, attestation = line.split("\t")
       # We use "\t" as the "start sequence" character and "\n" as "end sequence" character for the target text.
      input.append(i_text)
      target.append("\t"+t_text+"\n")
      for char in i_text:
        if char not in input_chars:
            input_chars.add(char)
      for char in t_text:
        if char not in target_chars:
            target_chars.add(char)
  target_chars.add("\t")
  target_chars.add("\n")

  input_chars = sorted(list(input_chars))
  target_chars = sorted(list(target_chars))
  num_encoder_tokens = len(input_chars)
  num_decoder_tokens = len(target_chars)
  max_encoder_seq_length = max([len(txt) for txt in input])
  max_decoder_seq_length = max([len(txt) for txt in target])
  return input,target,input_chars,target_chars,num_encoder_tokens,num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length     

In [7]:
# Vectorize the data.
input,target,input_chars,target_chars,num_encoder_tokens,num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length=processData(train_file_path)
print("Number of samples:", len(input))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 44204
Number of unique input tokens: 26
Number of unique output tokens: 65
Max sequence length for inputs: 20
Max sequence length for outputs: 21


In [8]:
# Vectorize the data.
# Vectorize the data.
validation_input,validation_target,input_chars,target_chars,num_encoder_tokens,num_decoder_tokens, validation_max_encoder_seq_length, validation_max_decoder_seq_length=processData(val_file_path,set(input_chars),set(target_chars))

print("Number of validation samples:", len(validation_input))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("validation Max sequence length for inputs:", validation_max_encoder_seq_length)
print("validation Max sequence length for outputs:", validation_max_decoder_seq_length)

Number of validation samples: 4502
Number of unique input tokens: 26
Number of unique output tokens: 65
validation Max sequence length for inputs: 16
validation Max sequence length for outputs: 17


In [9]:
# Vectorize the data.
test_input,test_target,test_input_chars,test_target_chars,test_num_encoder_tokens,test_num_decoder_tokens, test_max_encoder_seq_length, test_max_decoder_seq_length=processData(test_file_path)
print("Number of validation samples:", len(test_input))
print("Test Max sequence length for inputs:", test_max_encoder_seq_length)
print("Test Max sequence length for outputs:", test_max_decoder_seq_length)

Number of validation samples: 4358
Test Max sequence length for inputs: 18
Test Max sequence length for outputs: 16


In [10]:
input_token = dict([(char, i) for i, char in enumerate(input_chars)])
target_token = dict([(char, i) for i, char in enumerate(target_chars)])

reverse_input_token = dict((i, char) for char, i in input_token.items())
reverse_target_token = dict((i, char) for char, i in target_token.items())


encoder_input_data = np.zeros(
    (len(input), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
validation_encoder_input_data=np.zeros(
    (len(validation_input), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
test_encoder_input_data=np.zeros(
    (len(test_input), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
validation_decoder_input_data =np.zeros(
    (len(validation_input), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
validation_decoder_target_data = np.zeros(
    (len(validation_input), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input, target)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token[char]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token[char]] = 1.0
# for validation data
for i, (validation_input_text, validation_target_text) in enumerate(zip(validation_input, validation_target)):
    for t, char in enumerate(validation_input_text):
        validation_encoder_input_data[i, t, input_token[char]] = 1.0
    for t, char in enumerate(validation_target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        validation_decoder_input_data[i, t, target_token[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            validation_decoder_target_data[i, t - 1, target_token[char]] = 1.0

# for test data
for i, (test_input_text, test_target_text) in enumerate(zip(test_input, test_target)):
    for t, char in enumerate(test_input_text):
        test_encoder_input_data[i, t, input_token[char]] = 1.0

In [120]:
input_token = dict([(char, i) for i, char in enumerate(input_chars)])
target_token = dict([(char, i) for i, char in enumerate(target_chars)])

reverse_input_token = dict((i, char) for char, i in input_token.items())
reverse_target_token = dict((i, char) for char, i in target_token.items())

enc_input_data = np.zeros(
    (len(input), max_encoder_seq_length), dtype="float32"
)
dec_input_data = np.zeros(
    (len(input), max_decoder_seq_length), dtype="float32"
)
dec_target_data = np.zeros(
    (len(input), max_decoder_seq_length), dtype="float32"
)
#Decoder Target Sequences are Padded to a maximum length of max_decoder SeqLen characters with a vocabulary of sizeofTeluguVocab different characters. 
for i, (input_text, target_text) in enumerate(zip(input, target)):
    for t, char in enumerate(input_text):
        enc_input_data[i, t] = input_token[char]
    #enc_input_data[i, t + 1 :] = input_token[" "]

    for t, char in enumerate(target_text):
        # dec_target_data is ahead of dec_input_data by one timestep
        dec_input_data[i, t] = target_token[char]
        if t > 0:
            # dec_target_data will not include the start character.
            dec_target_data[i, t - 1] = target_token[char]
    #dec_input_data[i, t + 1: ] = target_token[" "]
    #dec_target_data[i, t:, target_token[" "]] = 1.0
    
val_enc_input_data = np.zeros(
    (len(validation_input), validation_max_encoder_seq_length), dtype="float32"
)
val_dec_input_data = np.zeros(
    (len(validation_input), validation_max_decoder_seq_length), dtype="float32"
)
val_dec_target_data = np.zeros(
    (len(validation_input), validation_max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(validation_input,validation_target)):
    for t, char in enumerate(input_text):
        # Enumerate() method adds a counter to an iterable and returns it in a form of enumerating object. 
        # This enumerated object can then be used directly for loops or converted into a list of tuples using the list() method.
        val_enc_input_data[i, t] = input_token[char]
    #val_enc_input_data[i, t + 1 :] = input_token_index[" "]

    for t, char in enumerate(target_text):
        val_dec_input_data[i, t] = target_token[char]
        if t > 0:
            # dec_target_data will be ahead by one timestep
            # and will not include the start character.
            val_dec_target_data[i, t - 1, target_token[char]] = 1.0
    #val_dec_input_data[i, t + 1: ] = target_token_index[" "]

In [133]:
class NMTDataset:
    def __init__(self, problem_type='en-hi'):
        self.problem_type = 'en-'
        self.inp_lang_tokenizer = None
        self.targ_lang_tokenizer = None
    

    def unicode_to_ascii(self, s):
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

    ## Step 1 and Step 2 
    def preprocess_sentence(self, w):
        # w = self.unicode_to_ascii(w.lower().strip())

        # # creating a space between a word and the punctuation following it
        # # eg: "he is a boy." => "he is a boy ."
        # # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
        # w = re.sub(r"([?.!,¿])", r" \1 ", w)
        # w = re.sub(r'[" "]+', " ", w)

        # # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
        # w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

        # w = w.strip()

        # adding a start and an end token to the sentence
        # so that the model know when to start and stop predicting.
        #print(w)
        w = '\t' + w + '\n'
        
        return w
    
    def create_dataset(self, path, num_examples):
        # path : path to spa-eng.txt file
        # num_examples : Limit the total number of training example for faster training (set num_examples = len(lines) to use full data)
        #lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
        #word_pairs = [[self.preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
        data =  pd.read_csv(path,delimiter="\t", header= None, nrows = num_examples )
        data = data.dropna()
        print(data.info())
        return data[0].apply(self.preprocess_sentence).values.astype(str), data[1].apply(self.preprocess_sentence).values.astype(str)

    # Step 3 and Step 4
    def tokenize(self, lang):
        # lang = list of sentences in a language
        
        # print(len(lang), "example sentence: {}".format(lang[0]))
        lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level = True)
        lang_tokenizer.fit_on_texts(lang)

        ## tf.keras.preprocessing.text.Tokenizer.texts_to_sequences converts string (w1, w2, w3, ......, wn) 
        ## to a list of correspoding integer ids of words (id_w1, id_w2, id_w3, ...., id_wn)
        tensor = lang_tokenizer.texts_to_sequences(lang) 

        ## tf.keras.preprocessing.sequence.pad_sequences takes argument a list of integer id sequences 
        ## and pads the sequences to match the longest sequences in the given input
        tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

        return tensor, lang_tokenizer

    def load_dataset(self, path, num_examples=None):
        # creating cleaned input, output pairs
        targ_lang, inp_lang = self.create_dataset(path, num_examples)

        input_tensor, inp_lang_tokenizer = self.tokenize(inp_lang)
        target_tensor, targ_lang_tokenizer = self.tokenize(targ_lang)

        return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

    def call(self, num_examples, BUFFER_SIZE, BATCH_SIZE):
        #file_path = download_dakshina()
        input_tensor_train, target_tensor_train, self.inp_lang_tokenizer, self.targ_lang_tokenizer = self.load_dataset(train_file_path, num_examples)
        input_tensor_val, target_tensor_val, self.inp_lang_tokenizer, self.targ_lang_tokenizer = self.load_dataset(val_file_path, num_examples)
        input_tensor_test, target_tensor_test, self.inp_lang_tokenizer, self.targ_lang_tokenizer = self.load_dataset(test_file_path, num_examples)
        x = input_tensor_train
        y  =target_tensor_train
        #input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.4)

        train_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train,target_tensor_train))
        train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

        val_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val))
        val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

        test_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_test, target_tensor_test))
        test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)
        return train_dataset, val_dataset, test_dataset, self.inp_lang_tokenizer, self.targ_lang_tokenizer


In [134]:
BUFFER_SIZE = 32000
BATCH_SIZE = 64
# Let's limit the #training examples for faster training
num_examples = 300000

dataset_creator = NMTDataset('en-spa')
train_dataset, val_dataset,test_dataset, inp_lang, targ_lang = dataset_creator.call(num_examples, BUFFER_SIZE, BATCH_SIZE)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44202 entries, 0 to 44203
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       44202 non-null  object
 1   1       44202 non-null  object
 2   2       44202 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4502 entries, 0 to 4501
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       4502 non-null   object
 1   1       4502 non-null   object
 2   2       4502 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 140.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4358 entries, 0 to 4357
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       4358 non-null   object
 1   1       4358 non-null   object
 2   2       4358 non-null   int64 
dtypes: int64(1), object(2)


In [144]:
next(iter(train_dataset))[1][0]

<tf.Tensor: shape=(21,), dtype=int32, numpy=
array([ 1,  7, 15, 28,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0], dtype=int32)>

In [66]:
example_input_batch, example_target_batch = next(iter(train_dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 22]), TensorShape([64, 21]))

## Build the model


In [35]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, num_of_layers, enc_unit_type, batch_sz, recurrent_dropout, dropout):
    super(Encoder, self).__init__()

    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.enc_unit_type = enc_unit_type
    self.num_of_layers = num_of_layers
    self.recurrent_dropout = recurrent_dropout
    self.dropout = dropout
    self.embedding = Embedding( vocab_size, embedding_dim)

    self.encoder_layer = self.get_encoder_layer(self.enc_units,
                                                self.num_of_layers, self.enc_unit_type)
    

  def get_encoder_layer(self, enc_units, num_of_layers, enc_unit_type):
    return tf.keras.layers.RNN(tf.keras.layers.StackedRNNCells( [self.get_cell(enc_unit_type, 
                                                                                 enc_units) for i in range(num_of_layers)],),
                                  return_sequences=True, return_state=True, name = "Encoder")

  def get_cell(self, cell_type = "lstm", num_of_cell = 1, name = None):
      #print(cell_type)
      if cell_type == "lstm":
        return LSTMCell(num_of_cell, dropout = self.dropout, recurrent_dropout = self.recurrent_dropout, )
      elif cell_type == "rnn":
        return SimpleRNNCell(num_of_cell, dropout = self.dropout, recurrent_dropout = self.recurrent_dropout)
      elif cell_type =="gru":
        return GRUCell(num_of_cell, dropout = self.dropout, recurrent_dropout = self.recurrent_dropout)
      else:
        print(f"Invalid cell type: {cell_type}")

    
  def call(self, x):
      x = self.embedding(x)
      output = self.encoder_layer(x,)

      #print(output)
      return output
    
  def initialize_hidden_state(self):
      print("Called")
      return [tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units))]

In [278]:
enc_input_data[0].shape

(20,)

In [501]:

#encoder = Encoder( num_encoder_tokens, 1024, 1024, 3, "lstm", batch_size, 0.0, 0.0).build(input_shape =(None,22))
encoder = Encoder( num_encoder_tokens, 1, 16, 3, "lstm", 64, 0.0, 0.0)
#sample_hidden = encoder.initialize_hidden_state()
# encoder.build(input_shape =(None,26))
# encoder.summary()
sample_output = encoder(enc_input_data[:64])
out , state = sample_output[0], sample_output[1:]

In [ ]:
out, state

In [17]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, num_of_layers, 
               dec_unit_type, batch_sz, recurrent_dropout, dropout, 
               attention_type = "luong"):
    
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.dec_unit_type = dec_unit_type
    self.num_of_layers = num_of_layers
    self.attention_type = attention_type
    self.recurrent_dropout = recurrent_dropout
    self.dropout = dropout
    print("decoder embedding dim", embedding_dim)
    self.embedding = Embedding( vocab_size, embedding_dim)

    self.fc  = tf.keras.layers.Dense(vocab_size)

    self.decoder_cells = self.get_stacked_rnn_cell()

    self.sampler = tfa.seq2seq.sampler.TrainingSampler()
    self.attention_mechanism = self.build_attention_mechanism(self.dec_units, None
                                                              , self.batch_sz*[max_encoder_seq_length], 
                                                              self.attention_type)

    self.cell = self.build_cell()

    #print(self.cell)

    self.decoder = tfa.seq2seq.BasicDecoder(self.cell, sampler = self.sampler, output_layer = self.fc)



  def build_cell(self):
    cell = tfa.seq2seq.AttentionWrapper(self.decoder_cells, self.attention_mechanism,
                                        attention_layer_size = self.dec_units)
    return cell
  
  def build_attention_mechanism(self, dec_units, memory, memory_sequence_length, attention_type='luong'):
    # ------------- #
    # typ: Which sort of attention (Bahdanau, Luong)
    # dec_units: final dimension of attention outputs 
    # memory: encoder hidden states of shape (batch_size, max_length_input, enc_units)
    # memory_sequence_length: 1d array of shape (batch_size) with every element set to max_length_input (for masking purpose)

    if(attention_type=='bahdanau'):
      return tfa.seq2seq.BahdanauAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)
    else:
      return tfa.seq2seq.LuongAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)

  def get_cell(self, cell_type = "lstm", num_of_cell = 1, name = None):
      #print(cell_type)
      if cell_type == "lstm":
        return LSTMCell(num_of_cell, dropout = self.dropout, recurrent_dropout = self.recurrent_dropout, )
      elif cell_type == "rnn":
        return SimpleRNNCell(num_of_cell, dropout = self.dropout, recurrent_dropout = self.recurrent_dropout)
      elif cell_type =="gru":
        return GRUCell(num_of_cell, dropout = self.dropout, recurrent_dropout = self.recurrent_dropout)
      else:
        print(f"Invalid cell type: {cell_type}")

  def get_stacked_rnn_cell(self,):
    return tf.keras.layers.StackedRNNCells( [self.get_cell(self.dec_unit_type, self.dec_units,) for i in range(self.num_of_layers)])

  def build_initial_state(self, batch_sz, encoder_state, Dtype):
    decoder_initial_state = self.cell.get_initial_state(batch_size=batch_sz, dtype=Dtype)
    #print(batch_sz)
    #print(len(encoder_state))
    decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)
    return decoder_initial_state

  def call(self, x, initial_state):
    x = self.embedding(x)
    print("calles")
    output = self.decoder(x, initial_state=initial_state)
    return output

In [502]:

#encoder = Encoder( num_encoder_tokens, 1024, 1024, 3, "lstm", batch_size, 0.0, 0.0).build(input_shape =(None,22))
decoder = Decoder( num_decoder_tokens,  1, 16, 3, "lstm", 64, 0.0, 0.0)
#sample_hidden = encoder.initialize_hidden_state()
#decoder.build(input_shape =(None, ))
# decoder.summary()
#sample_x = tf.random.uniform((2  ,max_decoder_seq_length))
decoder.attention_mechanism.setup_memory(out)
initial_state = decoder.build_initial_state(64, tuple(state), tf.float32)
# sample_output = decoder(dec_input_data[:8192], initial_state)
# out1 , state1 = sample_output[0], sample_output[1:]

decoder embedding dim 1


In [ ]:
out1

In [95]:
class seq2seq(tf.keras.Model):
  def __init__(self, num_encoder_tokens, num_decoder_token, encoder_embedding_dim, decoder_embedding_dim,num_of_unit, num_of_layers, unit_type, batch_size, recurrent_dropout, dropout):
    super().__init__()
    self.batch_size = batch_size
    self.encoder = Encoder(  num_encoder_tokens, 1, 16, 3, "lstm", self.batch_size, 0.0, 0.0)
    #self.encoder.summary()
    self.dec = Decoder( num_decoder_tokens,  1, 16, 3, "lstm", self.batch_size, 0.0, 0.0)
    #sample_x = tf.random.uniform((batch_size  ,max_decoder_seq_length))

  def call(self, enc_inp, dec_inp):
    print("fsdfa",dec_inp.shape)
    x = self.encoder(enc_inp)
    enc_out, enc_state = x[0], x[1:]
    print(enc_out.shape)
    self.dec.attention_mechanism.setup_memory(enc_out)
    dec_initial_state = self.dec.build_initial_state(self.batch_size, tuple(enc_state), tf.float32)
    print("fucck")
    x = self.dec(dec_inp,dec_initial_state)
    return x



    

In [124]:
batch_size = 64
s2s = seq2seq(num_encoder_tokens, num_decoder_tokens,  encoder_embedding_dim =256,
              decoder_embedding_dim= 1024,
              num_of_unit =16,
              num_of_layers = 5, 
              unit_type ="gru",
             batch_size = batch_size, 
              recurrent_dropout = 0,
              dropout = 0 )
sample_out = s2s(enc_input_data[:batch_size], dec_input_data[:batch_size])

decoder embedding dim 1
fsdfa (64, 21)
(64, 20, 16)
fucck
calles


In [128]:
cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
loss = cross_entropy(y_true=dec_target_data[:64], y_pred=sample_out[0].rnn_output)
loss

<tf.Tensor: shape=(64, 21), dtype=float32, numpy=
array([[4.1747932, 4.1745915, 4.174487 , ..., 4.174175 , 4.174177 ,
        4.17418  ],
       [4.174734 , 4.174542 , 4.1742854, ..., 4.1741757, 4.174185 ,
        4.1741924],
       [4.174767 , 4.1745205, 4.1742883, ..., 4.174209 , 4.174211 ,
        4.174213 ],
       ...,
       [4.1746445, 4.1743155, 4.174321 , ..., 4.1742435, 4.174258 ,
        4.1742716],
       [4.174657 , 4.174332 , 4.1743355, ..., 4.174272 , 4.174281 ,
        4.1742883],
       [4.174667 , 4.1743417, 4.174344 , ..., 4.1742654, 4.174274 ,
        4.1742816]], dtype=float32)>

In [130]:
loss_function(dec_target_data[:64], sample_out[0].rnn_output)

<tf.Tensor: shape=(), dtype=float32, numpy=1.3852632>

In [112]:
optimizer = tf.keras.optimizers.Adam()


def loss_function(real, pred):
  # real shape = (BATCH_SIZE, max_length_output)
  # pred shape = (BATCH_SIZE, max_length_output, tar_vocab_size )
  #print(pred,"fucck", real)
  cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
  loss = cross_entropy(y_true=real, y_pred=pred)
  mask = tf.logical_not(tf.math.equal(real,0))   #output 0 for y=0 else output 1
  mask = tf.cast(mask, dtype=loss.dtype)  
  loss = mask* loss
  loss = tf.reduce_mean(loss)
  return loss  

In [113]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=s2s.encoder,
                                 decoder=s2s.dec,
                                 seq2seq = s2s)

In [132]:
 dec_input_data[0]

array([0., 5., 3., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.], dtype=float32)

In [146]:
@tf.function
def train_step(enc_input_data, dec_input_data, targ):
  loss = 0

  with tf.GradientTape() as tape:
    print("from tf.functio",enc_input_data.shape, dec_input_data.shape, targ.shape)
    dec_input_data = dec_input_data[ : , :-1 ]
    out = s2s(enc_input_data, dec_input_data)
    #enc_output, enc_h, enc_c = encoder(inp, enc_hidden)


    dec_input = targ[ : , :-1 ] # Ignore <end> token
    real = targ[ : , 1: ]         # ignore <start> token

    # Set the AttentionMechanism object with encoder_outputs
    #decoder.attention_mechanism.setup_memory(enc_output)

    # Create AttentionWrapperState as initial_state for decoder
    #decoder_initial_state = decoder.build_initial_state(BATCH_SIZE, [enc_h, enc_c], tf.float32)
    #pred = decoder(dec_input, decoder_initial_state)
    logits = out[0].rnn_output
    #print(logits.item())
    loss = loss_function(real, logits)

  variables = s2s.variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))

  return loss

In [147]:
EPOCHS = 10
steps_per_epoch = len(input)//batch_size
print(steps_per_epoch)
input_data = tf.data.Dataset.from_tensor_slices((enc_input_data, dec_input_data))
target_data =  tf.data.Dataset.from_tensor_slices(dec_target_data)
train_dataset  = tf.data.Dataset.zip((input_data, target_data)).batch(batch_size)
for epoch in range(EPOCHS):
  start = time.time()

  #enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
  # print(enc_hidden[0].shape, enc_hidden[1].shape)


  for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp[0],inp[1] ,targ )
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

690
from tf.functio (64, 20) (64, 21) (64, 21)
fsdfa (64, 20)
(64, 20, 16)
fucck
calles
from tf.functio (64, 20) (64, 21) (64, 21)
fsdfa (64, 20)
(64, 20, 16)
fucck
calles
Epoch 1 Batch 0 Loss 1.2458
Epoch 1 Batch 100 Loss 0.9433
Epoch 1 Batch 200 Loss 0.8726
Epoch 1 Batch 300 Loss 0.7946
Epoch 1 Batch 400 Loss 0.8270
Epoch 1 Batch 500 Loss 1.1725
Epoch 1 Batch 600 Loss 0.8587
Epoch 1 Loss 1.0207
Time taken for 1 epoch 33.06755042076111 sec

Epoch 2 Batch 0 Loss 0.9792
Epoch 2 Batch 100 Loss 0.9401
Epoch 2 Batch 200 Loss 0.8784
Epoch 2 Batch 300 Loss 0.7667
Epoch 2 Batch 400 Loss 0.7683
Epoch 2 Batch 500 Loss 1.1255
Epoch 2 Batch 600 Loss 0.8203
Epoch 2 Loss 0.9704
Time taken for 1 epoch 25.664228677749634 sec

Epoch 3 Batch 0 Loss 0.9330
Epoch 3 Batch 100 Loss 0.9260
Epoch 3 Batch 200 Loss 0.8411
Epoch 3 Batch 300 Loss 0.7536
Epoch 3 Batch 400 Loss 0.7239
Epoch 3 Batch 500 Loss 1.1069
Epoch 3 Batch 600 Loss 0.7989
Epoch 3 Loss 0.9435
Time taken for 1 epoch 24.51833748817444 sec

Epoch

In [153]:
s2s.predict([val_enc_input_data,val_dec_input_data])

ValueError: ignored

In [152]:
s2s.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=[tf.keras.metrics.CategoricalAccuracy(name='acc')]
)

#s2s.summary()
s2s.fit(
    train_dataset,
    batch_size=64,
    epochs=1,
)

ValueError: ignored

In [40]:

class Seq2seq(tf.keras.Model):
  def __init__(self, num_encoder_tokens, num_decoder_tokens,embedding_dim,num_of_layers,unit_type, dropout , recurrent_dropout):
    super().__init__()
    self.encoder_inputs = Input(shape = (None,), name = "Input_layer_1")
    self.decoder_inputs = keras.Input(shape=(None,), name = "Input_layer_2")
    self.num_encoder_tokens = num_encoder_tokens
    self.embedding_dim = embedding_dim
    self.dropout = dropout
    self.recurrent_dropout = recurrent_dropout
    self.num_decoder_tokens = num_decoder_tokens
    self.num_of_encoder_layer  =num_of_layers
    self.num_of_decoder_layer =num_of_layers
    self.type_encoder_unit =unit_type 
    self.type_decoder_unit =unit_type
    self.train_step()
    self.build_model()

  def get_embedding_layer(self, num_encoder_tokens, embedding_dim,  name):
    return Embedding(num_encoder_tokens, embedding_dim, mask_zero = True, name =name )

  def get_cell(self, cell_type = "lstm", num_of_cell = 1, name = None):
    #print(cell_type)
    if cell_type == "lstm":
      return LSTMCell(num_of_cell, dropout = self.dropout, recurrent_dropout = self.recurrent_dropout, )
    elif cell_type == "rnn":
      return SimpleRNNCell(num_of_cell, dropout = self.dropout, recurrent_dropout = self.recurrent_dropout)
    elif cell_type =="gru":
      return GRUCell(num_of_cell, dropout = self.dropout, recurrent_dropout = self.recurrent_dropout)
    else:
      print(f"Invalid cell type: {cell_type}")
  def get_encoder(self,latent_dim, cell_type = "lstm", num_of_layer = 1, name = None ):
    return tf.keras.layers.RNN(tf.keras.layers.StackedRNNCells( [self.get_cell(cell_type, latent_dim) for i in range(num_of_layer)],), return_sequences=True, return_state=True, name = name)

  def get_decoder(self,latent_dim ,cell_type = "lstm", num_of_layer = 1, name = None ):
    return tf.keras.layers.RNN(tf.keras.layers.StackedRNNCells( [self.get_cell(cell_type, latent_dim,) for i in range(num_of_layer)]), return_sequences=True, return_state=True)

  def get_dense_layer(self, num_decoder_token, activation = "softmax"):
    return Dense(num_decoder_tokens, activation= activation)

  def train_step(self):
    self.embedding_layer = self.get_embedding_layer( self.num_encoder_tokens, self.embedding_dim ,name = "encoder_embedding")
    self.embedding_results = self.embedding_layer(self.encoder_inputs)
    print(self.embedding_results.shape)
    self.encoder = self.get_encoder( self.embedding_dim,self.type_encoder_unit, self.num_of_encoder_layer , name ="encoder" )
    encoder_results = self.encoder(self.embedding_results)

    self.encoder_outputs, self.encoder_states = encoder_results[0], encoder_results[1:]

    self.embedding_layer2 = self.get_embedding_layer( self.num_decoder_tokens, self.embedding_dim, name = "decoder_embedding")
    self.embedding_results2 = self.embedding_layer2(self.decoder_inputs,)

    self.decoder = self.get_decoder( self.embedding_dim, self.type_decoder_unit, self.num_of_decoder_layer,)
    self.decoder_results = self.decoder(self.embedding_results2, initial_state=self.encoder_states)

    self.decoder_output = self.decoder_results[0]
    self.decoder_dense = self.get_dense_layer(self.num_decoder_tokens)
    self.dense_output = self.decoder_dense(self.decoder_output)

  def build_model(self):
    
    self.model = keras.Model([self.encoder_inputs, self.decoder_inputs], self.dense_output, name = "Seq2Seq_model")
    return self.model



In [14]:

seq2seq = Seq2seq(num_encoder_tokens,num_decoder_tokens, 1024,1,"rnn", 0.0, 0.0).build_model()
seq2seq.summary()

(None, None, 1024)
Model: "Seq2Seq_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input_layer_1 (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 Input_layer_2 (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, None, 1024)   26624       ['Input_layer_1[0][0]']          
                                                                                                  
 decoder_embedding (Embedding)  (None, None, 1024)   66560       ['Input_layer_2[0][0]']          
                                                                   

## Train the model


In [80]:
s2s.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=[tf.keras.metrics.CategoricalAccuracy(name='acc')]
)

s2s.fit(
    [enc_input_data, dec_input_data],
    dec_target_data,
    batch_size=64,
    epochs=1,
)

ValueError: ignored

In [15]:
class BeamSearch(keras.callbacks.Callback):

  def __init__(self, beam_size):
    self.beam_size = beam_size

  def beam_search_decoder(aelf, data, k):
    sequences = [[list(), 0.0]]
    # walk over each step in sequence
    for row in data:
      all_candidates = list()
      # expand each current candidate
      for i in range(len(sequences)):
        seq, score = sequences[i]
        for j in range(len(row)):
          candidate = [seq + [j], score - log(row[j])]
          all_candidates.append(candidate)
      # order all candidates by score
      ordered = sorted(all_candidates, key=lambda tup:tup[1])
      # select k best
      sequences = ordered[:k]
    return sequences
  
  def on_epoch_end(self, epoch, logs = None):
    prediction = self.model.predict([val_enc_input_data , val_dec_input_data])
    print(prediction.shape)
    for i, pred in enumerate(prediction):
      beam_search_prediction = self.beam_search_decoder(pred, self.beam_size)
      correct_prediction = 0
      for k in range(self.beam_size):
        #translated_word = "\t"+"".join([reverse_target_token[x] for x in beam_search_prediction[k][0][:len(validation_target[i])-1]])
        #print(translated_word, validation_target[i])
        #print(validation_target[i])
        
        def idx2char(idx_list):
          return "".join([reverse_target_token[x] for x in idx_list])

        if "\t"+ idx2char(beam_search_prediction[k][0][:len(validation_target[i])-1]) == validation_target[i]:
          correct_prediction+=1
          break
    mul = 10.0**2
    logs["character_accuracy"] = ((correct_prediction/prediction.shape[0])*mul)/mul
    print("- character_accuracy",logs["character_accuracy"])
    #print(f"Accuracy by Beam Search {correct_prediction/len(validation_target)}")
      # print(len(beam_search_prediction))
      # print(beam_search_prediction)


In [ ]:
def beam_search_decoder(data, k):
    decodedWords = [[list(), 0.0]]
    # walk over each step in sequence
    for word in data:
      candidates = list()
      # expand each current candidate
      for sequence in decodedWords:
        seq, score = sequence
        for j in range(len(word)):
          candidate = [seq + [j], score - log(word[j])]
          candidates.append(candidate)
      # order all candidates by score
      ordered = sorted(candidates, key=lambda a:a[1])
      # select k best
      decodedWords = ordered[:k]
    return decodedWords
  
def translate(seq):
  sentence = [] 
  for x in seq:
    char = reverse_target_token[x]
    sentence.append(char)
  return "".join(sentence)
class WordAccuracyCallback(keras.callbacks.Callback):
  def __init__(self,beam_size):
    self.beam_size=beam_size
  def on_epoch_end(self, epoch, logs=None):
    pred=self.model.predict([val_enc_input_data , val_dec_input_data])
    count=0
    for i in range(pred.shape[0]):
      pSequences=beam_search_decoder(pred[i],self.beam_size)
      for j in range(self.beam_size):
        if "\t"+translate(pSequences[j][0][:len(validation_target[i])-1])==validation_target[i]:
          count=count+1
          break
    factor = 10.0 ** 4
    logs["WordAccuracy"]=math.trunc((count/pred.shape[0])*factor)/factor
    print("- wordAccuracy:",logs["WordAccuracy"])

In [ ]:
sweep_config = {
    
    'method':'bayes',
    'metric': {
        'name':'val_accuracy',
        'goal':'maximize'
    },
    'parameters':{
    
    "num_of_layer" : {'values': [1,2,3]},
    "unit_size": {"values":[16,32,64]},
    "unit_type": {"values":["lstm","rnn","gru"]},
    "dropout": {"values": [0.0, 0.2, 0.4]},
    'recurrent_dropout':{'values':[0.0,0.3]},
    "beam_size" : {"values":[1,2,3,4]},
    "epochs":{"value":20},  
    "optimizer":{"values": ["adam","rmsprop"]}             
                   }
}



pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'val_accuracy'},
 'parameters': {'beam_size': {'values': [1, 2, 3, 4]},
                'dropout': {'values': [0.0, 0.2, 0.4]},
                'epochs': {'value': 20},
                'num_of_layer': {'values': [1, 2, 3]},
                'optimizer': {'values': ['adam', 'rmsprop']},
                'recurrent_dropout': {'values': [0.0, 0.3]},
                'unit_size': {'values': [16, 32, 64]},
                'unit_type': {'values': ['lstm', 'rnn', 'gru']}}}


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="seq2seq")

In [ ]:
def train(config = None):
  with wandb.init(config=config):
    config = wandb.config
    #print(config)
    seq2seq = Seq2seq(num_encoder_tokens,num_decoder_tokens, config.unit_size, config.num_of_layer,config.unit_type , config.dropout,config.recurrent_dropout).build_model()
    seq2seq.compile(optimizer=config.optimizer, loss="categorical_crossentropy", metrics=["accuracy",])
    seq2seq.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size=batch_size,
        epochs=config.epochs,
        validation_data =  ([validation_encoder_input_data , validation_decoder_input_data] ,validation_decoder_target_data),
        callbacks = [BeamSearch(config.beam_size), WandbCallback()],verbose = 1, 
        )


    
    
wandb.agent(sweep_id, train)

In [ ]:
seq2seq.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=[tf.keras.metrics.CategoricalAccuracy(name='acc')]
)
seq2seq.metrics_names



[]

In [ ]:
pred=seq2seq.predict([val_enc_input_data , val_dec_input_data])
count=0
for i in range(pred.shape[0]//400):
      pSequences=beam_search_decoder(pred[i],3)
      for j in range(3):
        print({"\t"+translate(pSequences[j][0][:len(validation_target[i])-1])}, "original =", {validation_target[i]} )
        if "\t"+translate(pSequences[j][0][:len(validation_target[i])-1])==validation_target[i]:
          count=count+1
          print("yes")
          break
factor = 10.0 ** 4


{'\tकं\nा'} original = {'\tअंक\n'}
{'\tवं\nा'} original = {'\tअंक\n'}
{'\t्ं\nा'} original = {'\tअंक\n'}
{'\tकं\nा'} original = {'\tअंक\n'}
{'\tवं\nा'} original = {'\tअंक\n'}
{'\t्ं\nा'} original = {'\tअंक\n'}
{'\tकन\n\n\n\n'} original = {'\tअंकित\n'}
{'\tवन\n\n\n\n'} original = {'\tअंकित\n'}
{'\t्न\n\n\n\n'} original = {'\tअंकित\n'}
{'\tकन\n\n\n\n'} original = {'\tअंकों\n'}
{'\tवन\n\n\n\n'} original = {'\tअंकों\n'}
{'\t्न\n\n\n\n'} original = {'\tअंकों\n'}
{'\tकन\n\n\n\n'} original = {'\tअंकों\n'}
{'\tवन\n\n\n\n'} original = {'\tअंकों\n'}
{'\t्न\n\n\n\n'} original = {'\tअंकों\n'}
{'\tकन\n\n\n\n'} original = {'\tअंकों\n'}
{'\tवन\n\n\n\n'} original = {'\tअंकों\n'}
{'\t्न\n\n\n\n'} original = {'\tअंकों\n'}
{'\tकन\n\n\n\n'} original = {'\tअंकोर\n'}
{'\tवन\n\n\n\n'} original = {'\tअंकोर\n'}
{'\t्न\n\n\n\n'} original = {'\tअंकोर\n'}
{'\tकन\n\n\n\n'} original = {'\tअंकोर\n'}
{'\tवन\n\n\n\n'} original = {'\tअंकोर\n'}
{'\t्न\n\n\n\n'} original = {'\tअंकोर\n'}
{'\tकन\n\n\n\n\n'} original = {'\t

In [ ]:
x = seq2seq.predict([val_enc_input_data , val_dec_input_data])
x.shape

(4502, 17, 65)

In [ ]:

histotry = seq2seq.fit(
    [enc_input_data, dec_input_data],
    dec_target_data,
    batch_size=8192,
    epochs=1,
    callbacks = [WordAccuracyCallback(3), ],
)
# Save model
seq2seq.save("s2s")


6/6 [==============================] - 214s 34s/step - loss: 1.1271 - acc: 0.1550
- wordAccuracy: 0.0


In [ ]:
# for key in histotry.history.keys():
#       print(key , histotry.history[key])
#       #wandb.log({key : histotry.history[key]})

In [ ]:
# seq2seq.metrics_names

## Run inference (sampling)

1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states


In [ ]:
# # Define sampling models
# # Restore the model and construct the encoder and decoder.
# model = keras.models.load_model("s2s")

# encoder_inputs = model.input[0]  # input_1
# temp = model.layers[2].output
# encoder_outputs, state = temp[0], temp[1:]  # lstm_1
# encoder_states = state
# encoder_model = keras.Model(encoder_inputs, encoder_states)

# decoder_inputs = model.input[1]  # input_2
# decoder_state_input_h = keras.Input(shape=(latent_dim,))
# decoder_state_input_c = keras.Input(shape=(latent_dim,))
# decoder_states_inputs = state
# decoder_lstm = model.layers[3]
# temp = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# decoder_outputs, state_dec = temp[0], temp[1:]
# decoder_states = state_dec
# decoder_dense = model.layers[4]
# decoder_outputs = decoder_dense(decoder_outputs)
# decoder_model = keras.Model(
#     [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
# )

# # Reverse-lookup token index to decode sequences back to
# # something readable.
# # reverse_input_char_index = dict((i, char) for char, i in num_encoder_tokens.items())
# # reverse_target_char_index = dict((i, char) for char, i in num_decoder_tokens.items())
# # print(reverse_input_char_index)
# # print(input_token_index)

# reverse_input_token = dict((i, char) for char, i in input_token.items())
# reverse_target_token = dict((i, char) for char, i in target_token.items())
# def decode_sequence(input_seq):
#     # Encode the input as state vectors.
#     states_value = encoder_model.predict(input_seq)

#     # Generate empty target sequence of length 1.
#     target_seq = np.zeros((1, 1, num_decoder_tokens))
#     # Populate the first character of target sequence with the start character.
#     target_seq[0, 0, target_token_index["\t"]] = 1.0

#     # Sampling loop for a batch of sequences
#     # (to simplify, here we assume a batch of size 1).
#     stop_condition = False
#     decoded_sentence = ""
#     while not stop_condition:
#         temp = decoder_model.predict([target_seq] + states_value)
#         output_tokens, state = temp[0],temp[1:]

#         # Sample a token
#         sampled_token_index = np.argmax(output_tokens[0, -1, :])
#         #print(reverse_target_char_index)
#         sampled_char = reverse_target_token[sampled_token_index]
#         decoded_sentence += sampled_char

#         # Exit condition: either hit max length
#         # or find stop character.
#         if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
#             stop_condition = True

#         # Update the target sequence (of length 1).
#         target_seq = np.zeros((1, 1, num_decoder_tokens))
#         target_seq[0, 0, sampled_token_index] = 1.0

#         # Update states
#         states_value = state
#     return decoded_sentence



You can now generate decoded sentences as such:


In [ ]:
# for seq_index in range(20):
#     # Take one sequence (part of the training set)
#     # for trying out decoding.
#     input_seq = encoder_input_data[seq_index : seq_index + 1]
#     decoded_sentence = decode_sequence(input_seq)
#     print("-")
#     print("Input sentence:", input_texts[seq_index])
#     print("Decoded sentence:", decoded_sentence)


In [ ]:
! git log

fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
